## GeoPandas Spatial Index
Basically an Rtree 



#### Dependancies

In [1]:
import geopandas
from shapely.geometry import box, Polygon, LineString, Point

### Adding Points

Adding points to the spatial index can be done in multiple ways using different spatial objects. We will be adding points in this assignment. I'll show you two different ways.

#### First Way

We use the parallel array approach with x values in one list and y values in another:

In [7]:
x = [-128,-127,-129,-124,-127]
y = [34,33,36,40,34]

gs1 = geopandas.GeoSeries(geopandas.points_from_xy(x, y))

#### Second Way

We use a geopandas data type `Point` and simply insert a list of them. 

In [35]:
import json
f = open("./data_files/example.geojson")
data = json.load(f)

points = []
for feature in data["features"]:
    if feature["geometry"]["type"] == "Point":
        points.append(feature["geometry"]["coordinates"])

cities = []
for point in points:
    cities.append(Point(point))

gs2 = geopandas.GeoSeries(cities)

### Querying

To query the first structure we can create a bounding box of this form:
 
`box(minx,miny,maxx,maxy)`

In [23]:
gs1.sindex.query(box(-128, 33, -127, 34))

array([0, 1, 4])

#### GeoJson Rectangle to Bbox

In this example, I pulled a polygon from the `example.geojson` file, and wrote a snippet to turn it into a bounding box. This polygon represents a rectangle, but it still contains 5 points because of geojsons method of representing a rectangle. 

<img src="geojson_example2.png" width="200">

In [36]:
bbox = []
for feature in data["features"]:
    if "name" in feature["properties"] and feature["properties"]["name"]=="rectangleExample":
        for ring in feature["geometry"]["coordinates"]:
            for point in ring:
                bbox.append(point)
print("bbox:")
print(bbox)

minX = 99999
minY = 99999
maxX = -99999
maxY = -99999

for p in bbox:
    if p[0] < minX:
        minX = p[0]
    if p[1] < minY:
        minY = p[1]
    if p[0] > maxX:
        maxX = p[0]
    if p[1] > maxY:
        maxY = p[1]

result = gs2.sindex.query(box(minX, minY, maxX, maxY))

print("result:")
print(result)

print("City Results:")
for r in result:
    print(cities[r])

bbox:
[[-103.5791015625, 33.15594830078649], [-99.90966796875, 33.15594830078649], [-99.90966796875, 35.94243575255426], [-103.5791015625, 35.94243575255426], [-103.5791015625, 33.15594830078649]]
result:
[2 3]
City Results:
POINT (-101.865234375 33.5963189611327)
POINT (-101.84326171875 35.17380831799959)


#### GeoJson Polygon to GeoPandas Polygon

This is pretty much a 1 to 1 relationship, no conversion necessary. It's a list of points. 

If we just used the polygon from the geojson file (the one representing the rectangle), the result would be the same.

This is the same "polygon" as the example from above.

In [38]:
poly = []
for feature in data["features"]:
    if "name" in feature["properties"] and feature["properties"]["name"]=="rectangleExample":
        poly = feature["geometry"]["coordinates"]

# Remember a polygon is a "list of linestrings" so we are pulling the linestring out of the polygon object
poly = poly.pop()

result = gs2.sindex.query(Polygon(poly))

for r in result:
    print(cities[r])

POINT (-101.865234375 33.5963189611327)
POINT (-101.84326171875 35.17380831799959)


The following polygon is also from the `example.geojson` file shown in the image below:

<img src="geojson_example.png" width="200">

In [42]:
poly = []
for feature in data["features"]:
    if "name" in feature["properties"] and feature["properties"]["name"]=="polygonExample":
        for ring in feature["geometry"]["coordinates"]:
            poly = ring

result = gs2.sindex.query(Polygon(poly))

for r in result:
    print(cities[r])

POINT (-97.44873046875 35.26356186215209)
POINT (-97.3388671875 37.66642921209061)
POINT (-97.14111328125 33.19273094190692)
POINT (-95.99853515625 36.1733569352216)
POINT (-94.39453125 35.35321610123823)


### Getting Distance Between Points

The distance function returns the distance from (in this case) the point and every other spatial feature in the Rtree (geospatial index).

In [33]:
point = Point( -90.06591796875,29.99300228455108)
gs2.distance(point)

0     9.071092
1     8.566956
2    12.337251
3    12.866490
4     7.765092
5     6.889759
6    10.572477
dtype: float64

The above output shows the distance from the point parameter and every other item in the Rtree. Next we perform the same query, and find the single closest item to the point sent into the function (in this case New Orleans)

In [39]:
# grab the result in a list
result = gs2.distance(point)

# find the min index
min = 99999
index = -1
for i in range(len(result)):
    if result[i] < min:
        min = result[i]
        index = i

# print the closest city
print(cities[index])


POINT (-94.39453125 35.35321610123823)


For each point in the geoJson file (a city coordinate) find its closest neighbor. 

In [43]:
for point in cities:
    result = gs2.distance(point)

    # find the min index
    min = 99999
    index = -1
    for i in range(len(result)):
        if result[i] < min and result[i] > 0:
            min = result[i]
            index = i

    # print the closest city
    print(f"City: {point} => ClosesNeighbor: {cities[index]}")

City: POINT (-97.44873046875 35.26356186215209) => ClosesNeighbor: POINT (-95.99853515625 36.1733569352216)
City: POINT (-95.99853515625 36.1733569352216) => ClosesNeighbor: POINT (-97.44873046875 35.26356186215209)
City: POINT (-101.865234375 33.5963189611327) => ClosesNeighbor: POINT (-101.84326171875 35.17380831799959)
City: POINT (-101.84326171875 35.17380831799959) => ClosesNeighbor: POINT (-101.865234375 33.5963189611327)
City: POINT (-97.14111328125 33.19273094190692) => ClosesNeighbor: POINT (-97.44873046875 35.26356186215209)
City: POINT (-94.39453125 35.35321610123823) => ClosesNeighbor: POINT (-95.99853515625 36.1733569352216)
City: POINT (-97.3388671875 37.66642921209061) => ClosesNeighbor: POINT (-95.99853515625 36.1733569352216)
